___
## Ciência dos Dados - PROJETO 3 - INSPER

___
#### Freddy Dratwa

#### Victor Vazquez

#### Michel Haggiag

#### Allan Singer

#### Ricardo Ferraro

___

# Estudo do preço do carro

___

## A. INTRODUÇÃO

## https://www.kaggle.com/jpayne/852k-used-car-listings

O objetivo deste projeto é chegar ao melhor modelo possível que seja capaz de prever o preço de um carro usado nos EUA a partir das características presentes no dataset:
    - Preço;
    - Ano;
    - Quilometragem;
    - Cidade;
    - Estado;
    - Marca;
    - Modelo;
    - Número de Chassis.

A base de dados possui 1.2 milhão de carros com data de produção entre os anos 1997 a 2018 de 58 marcas diferentes, sendo estes 2914 modelos distintos com o preço mÍnimo de US1.500,00 e máximo de US462.000,00 , com a mínima quilometragem 8 Km e a máxima 2.286.260,8 Km.<br>

Como uma primeira previsão, imagina-se que algumas das características do dataset influenciam mais no preço do carro do que outras. Por exemplo, é de se esperar que um carro comprado há mais tempo tenha seu preço menor atualmente. Ou seja, a variavel "Ano", que representa o ano em que o carro foi fabricado, deve influenciar muito no preço do veículo, bem como a quilometragem, onde um carro que foi comprado mas usado pouco deve permanecer mais barato do que um usado muitas vezes.<br>

Pelo outro lado, imagina-se que a variável "Cidade", a qual representa a cidade onde o carro foi comprado, não tenha tanta influência no preço do veículo, a não ser que comparadas  duas cidades com condições muito diferentes. Por exemplo, como se tratam de cidades apenas nos EUA, imaina-se que em cidades no litoral, os carros se depreciem com velocidade maior, como consequência do fenômeno da Maresia.<br>

___
## B. MINERANDO DADOS e CARACTERÍSTICAS DO DATASET

In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split

### BASE DE DADOS

**Limpando e tratando dataset**

Nas células abaixo, o dataset é limpo e reorganiazdo. Para se fazer isso, a coluna "Chassi" é removoida, pois esta é uma característica individual para cada veículo e não influencia no preço final do mesmo. Alem disso, duas colunas vazias foram reitradas. Então, a variável milhagem foi transformada em quilometragem, multplicando seu valor por 1.6. Por último, a variavel ano foi transformada em idade, ou seja, a diferença do ano atual (2019) para o ano em que o carro foi vendido. 

In [2]:
dataset = pd.read_csv("Dataframe.csv")

In [3]:
if "Vin" in dataset:
    dataset = dataset.drop(columns=["Vin", "Unnamed: 9", "Unnamed: 10", "Unnamed: 0"])
    dataset["Mileage"] = dataset["Mileage"].apply(lambda x: x*1.6)

    dataset.columns = ["ID","PRECO","ANO","KM","CIDADE","ESTADO","MARCA","MODELO"]
    dataset["IDADE"] = dataset["ANO"].apply(lambda x: 2019-x)

    dataset.ID = dataset.ID.astype("float")
    dataset.PRECO = dataset.PRECO.astype("float")
    dataset.ANO = dataset.ANO.astype("float")
    dataset.KM = dataset.KM.astype("float")
    dataset.IDADE = dataset.IDADE.astype("float")
    dataset.CIDADE = dataset.CIDADE.astype("category")
    dataset.ESTADO = dataset.ESTADO.astype("category")
    dataset.MARCA = dataset.MARCA.astype("category")
    dataset.MODELO = dataset.MODELO.astype("category")

    dataset["ESTADO"] = dataset["ESTADO"].apply(lambda x: x.upper())
    dataset["CIDADE"] = dataset["CIDADE"].apply(lambda x: x.upper())
    dataset["MARCA"] = dataset["MARCA"].apply(lambda x: x.upper())
    dataset = dataset[["ID","PRECO","ANO","IDADE","KM","CIDADE","ESTADO","MARCA","MODELO"]]
    
dataset.head()

,ID,PRECO,ANO,IDADE,KM,CIDADE,ESTADO,MARCA,MODELO
0,79293.0,18684.0,2015.0,4.0,19998.4,OWENSBORO,KY,DODGE,JourneySXT
1,190834.0,26318.0,2016.0,3.0,13563.2,CORPUS CHRISTI,TX,BMW,3
2,670383.0,25802.0,2014.0,5.0,105404.8,PORT LAVACA,TX,FORD,ExplorerLimited
3,586732.0,9899.0,2005.0,14.0,173619.2,LINDON,UT,DODGE,Dakota4WD
4,37609.0,30999.0,2016.0,3.0,13372.8,FOND DU LAC,WI,FORD,MustangFastback


## ANÁLISE DESCRITIVA

___
## C. MODELOS DE PREDIÇÃO

o MODELO DE PREDIÇÃO PELA MÉDIA (Sem uso de variável explicativa).

**Separando a base de dados em treinamento e teste aleatoriamente**

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

In [4]:
def dummify(data, column_name):
    return pd.get_dummies(data[column_name],drop_first = False, prefix = column_name)

def regress(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    ypred = results.predict(X_cp)
    return results, ypred

In [5]:
final = pd.concat([dummify(dataset, "ESTADO").dropna(), dummify(dataset, "MODELO").dropna(), dummify(dataset, "MARCA").dropna()], axis=1)

In [6]:
dataset.head()

,ID,PRECO,ANO,IDADE,KM,CIDADE,ESTADO,MARCA,MODELO
0,79293.0,18684.0,2015.0,4.0,19998.4,OWENSBORO,KY,DODGE,JourneySXT
1,190834.0,26318.0,2016.0,3.0,13563.2,CORPUS CHRISTI,TX,BMW,3
2,670383.0,25802.0,2014.0,5.0,105404.8,PORT LAVACA,TX,FORD,ExplorerLimited
3,586732.0,9899.0,2005.0,14.0,173619.2,LINDON,UT,DODGE,Dakota4WD
4,37609.0,30999.0,2016.0,3.0,13372.8,FOND DU LAC,WI,FORD,MustangFastback


In [7]:
dt = dataset
dt = dt.drop(columns = ["ID", "ANO", "CIDADE", "ESTADO", "MARCA", "MODELO"])
final = pd.concat([dt, final], axis = 1)
final.head()

,PRECO,IDADE,KM,ESTADO_ AK,ESTADO_ AL,ESTADO_ AR,ESTADO_ AZ,ESTADO_ CA,ESTADO_ CO,ESTADO_ CT,...,MARCA_SAAB,MARCA_SATURN,MARCA_SCION,MARCA_SUBARU,MARCA_SUZUKI,MARCA_TESLA,MARCA_TOYOTA,MARCA_VOLKSWAGEN,MARCA_VOLVO,MARCA_SMART
0,18684.0,4.0,19998.4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,26318.0,3.0,13563.2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,25802.0,5.0,105404.8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9899.0,14.0,173619.2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,30999.0,3.0,13372.8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# np.delete(final.columns.values, 0)

In [9]:
df = final

x = df[np.delete(df.columns.values, 0)]
y = df[["PRECO"]]

results = regress(x,y)

dataresultados = pd.read_html(results[0].summary().tables[1].as_html(), header=0, index_col=0)[0]

In [69]:
dataresultados.head(10)

,coef,std err,t,P>|t|,[0.025,0.975]
const,51150.0000,249.667,204.860,0.000,50700.000,51600.000
IDADE,-1460.8541,9.277,-157.467,0.000,-1479.037,-1442.671
KM,-0.0421,0.000,-105.780,0.000,-0.043,-0.041
ESTADO_ AK,968.7678,392.679,2.467,0.014,199.122,1738.413
ESTADO_ AL,1191.2529,158.574,7.512,0.000,880.449,1502.056
ESTADO_ AR,1419.0408,200.383,7.082,0.000,1026.292,1811.790
ESTADO_ AZ,509.2999,135.975,3.746,0.000,242.790,775.810
ESTADO_ CA,651.7257,111.814,5.829,0.000,432.571,870.881
ESTADO_ CO,1161.6658,138.378,8.395,0.000,890.447,1432.884
ESTADO_ CT,37.2413,175.323,0.212,0.832,-306.390,380.872


In [ ]:
df = final
for i in range(5):
    df.drop(columns = dataresultados.loc[(dataresultados["P>|t|"] > 0.10)].index)
    x = df[np.delete(df.columns.values, 0)]
    y = df[["PRECO"]]

    results = regress(x,y)

    dataresultados = pd.read_html(results[0].summary().tables[1].as_html(), header=0, index_col=0)[0]
    
quanti = 0
for i in dataresultados["P>|t|"]:
    if i > 0.10:
        quanti+=1
print(quanti)

o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

___
## D. PROCESSO E ESTATÍSTICAS DE VALIDAÇÃO

___
## E. CONCLUSÃO

___
## F. REFERÊNCIAS BIBLIOGRÁFICAS